In [1]:
# ESTUDO DO PROBLEMA DE ESCALONAMENTO DE HORÁRIOS E ALGORITMOS GENÉTICOS

# Cada programa terá vários cursos
# Cada programa terá n semestres
# Cada curso pertencerá à um semestre especifico
# Cada semestre não poderá superar mais do que 10 aulas semanais no horario

# AG       -> Programa{ Cursos, Profs, Salas } : Schedule
# Programa -> {Horario Semanal para cada o programa com n semestres}

# *OBS1: Haverá "ataques ou choque de horarios" em um schedule para programa com n semestres (ataque entre semestres)
# *OBS2: Após a geração de um schedule pra um programa, guardar restrições desse programa para aplicar em outro, caso use os mesmos requisitos (salas e professores)


# ------------ AG ------------
# ___Code___
# Laço:
# - Popular
# - Fitness
# - Testar -> (Sol, S/ Sol, Evoluir)
# Fim

# Evoluir:
# - Seleção
# - Cruzamento
# - Mutação
# __________

# Conceitos:
# • Definir o que é cromossomo
# • Teste de Meta: Sim, se algum cromossomo tem fitness = 1. Não, caso o contrário.
# • Fitness: (1 - Erro/MaxErro)
#                  (Acerto/MaxAcerto)
# • Seleção: Roleta Viciada (Gera 2*n casais de cromossomos. Partição da roleta é baseada no valor Fitness)
# • Cruzamento: Cruzar casais gerados na Seleção. Ao cruzar, é pegada informação do macho e da fêmea, e são gerados 2 filhos melhores geneticamente.
# • Mutação: Mutar gene de cromossomos aleatoriamente

In [2]:
#Import .ipynb as module: %run name.ipynb
import math
import random
import copy
import itertools as it

In [3]:
### Data Structure for the Program Schedule Problem ###

# Macros
WEEK_SIZE         = 5
WEEK_CLASSES_SIZE = 10

# Class Course
class Course:
    _id = 0
    def __init__(self, name, classes, semester, teacherId=-1):
        Course._id     = Course._id +1
        self.id        = Course._id
        self.name      = name
        self.classes   = classes
        self.ammo      = classes
        self.semester  = semester
        self.teacherId = teacherId
        
    def __repr__(self):
        return 'Course {}: {} ({} classes, {}o semester, {} teacherId)'.format(self.id, self.name, self.classes, self.semester, self.teacherId)

# Class Teacher
class Teacher:
    _id = 0
    def __init__(self, name):
        Teacher._id  = Teacher._id +1
        self.id      = Teacher._id
        self.name    = name
        
    def __repr__(self):
        return 'Teacher {}: {}'.format(self.id, self.name)
    
# Class Classroom
class Classroom:
    _id = 0
    def __init__(self, name):
        Classroom._id = Classroom._id +1
        self.id       = Classroom._id
        self.name     = name
    
    def __repr__(self):
        return 'Classroom {}: {}'.format(self.id, self.name)

# Class Program
class Program:
    def __init__(self, name, semesters):
        self.name       = name
        self.semesters  = semesters
        self.courses    = []
        self.teachers   = []
        self.classrooms  = []
    
    def getCoursesBySemester(self, semester):
        coursesSemester = []
        for i in self.courses:
            if i.semester is semester and i.ammo > 0:
                coursesSemester.append(i)
        return coursesSemester
    
    def reloadCoursesAmmo(self):
        for i in self.courses:
            i.ammo = i.classes
            
    def addCourse(self, course):
        try:
            if course not in self.courses:
                if(sum([c.classes for c in self.getCoursesBySemester(course.semester)])+course.classes<=WEEK_CLASSES_SIZE):
                    self.courses.append(course)
        except:
            return
    
    def removeCourse(self, course):
        try:
            self.courses.remove(course)
        except:
            return
    
    def addTeacher(self, teacher):
        try:
            if teacher not in self.teachers:
                self.teachers.append(teacher)
        except:
            return
    
    def removeTeacher(self, teacher):
        try:
            self.teachers.remove(teacher)
        except:
            return
        
    def addClassroom(self, classroom):
        try:
            if classroom not in self.classrooms:
                self.classrooms.append(classroom)
        except:
            return
    
    def removeClassroom(self, classroom):
        try:
            self.classrooms.remove(classroom)
        except:
            return
        
    def __repr__(self):
        return 'Program: {} ({} semesters, {} courses, {} classrooms, {} teachers)'.format(self.name, self.semesters, len(self.courses), len(self.classrooms), len(self.teachers))

#Testes
# co1 = Course('Calculo I', 2, 1)
# co2 = Course('Calculo II', 2, 2)
# co3 = Course('Algebra Linear', 2, 2)
# co4 = Course('Algebra Linear 2', 4, 2)
# co5 = Course('Algebra Linear 3', 2, 2)
# co6 = Course('Algebra Linear 4', 1, 2)
# te1 = Teacher('Enio Romagnome')
# te2 = Teacher('Roberto Carlos')
# cl1 = Classroom('BC-23')
# cl2 = Classroom('BC-08')
# co1.teacherId = te1.id
# co2.teacherId = te2.id
# co3.teacherId = te2.id

In [4]:
### Input Data ###
# Program of EngComp
engComp = Program('EngComp', 10)
# 1 Semester
engComp.addCourse(Course('Lógica Matemática'                             ,2,1))
engComp.addCourse(Course('Introdução à Programação'                      ,3,1))
engComp.addCourse(Course('Eletrônica Digital'                            ,3,1))
engComp.addCourse(Course('Cálculo I'                                     ,2,1))
# 2 Semester
engComp.addCourse(Course('Matemática Discreta'                           ,2,2))
engComp.addCourse(Course('Programação Orientada à Objetos'               ,2,2))
engComp.addCourse(Course('Eletrônica Analógica'                          ,2,2))
engComp.addCourse(Course('Cálculo II'                                    ,2,2))
engComp.addCourse(Course('Física-Eletricidade'                           ,2,2))
# 3 Semester
engComp.addCourse(Course('Introdução à Análise de Algoritmos'            ,2,3))
engComp.addCourse(Course('Estrutura de Dados'                            ,2,3))
engComp.addCourse(Course('Circuitos Eletrônicos'                         ,2,3))
engComp.addCourse(Course('Arquitetura de Computadores'                   ,2,3))
engComp.addCourse(Course('Física-Eletromagnetismo'                       ,2,3))
# 4 Semester
engComp.addCourse(Course('Aspectos Teóricos da Computação'               ,2,4))
engComp.addCourse(Course('Pesquisa e Ordenação'                          ,2,4))
engComp.addCourse(Course('Microcontroladores e Microprocessadores'       ,3,4))
engComp.addCourse(Course('Geometria Analítica e Algebra Linear'          ,3,4))
# 5 Semester
engComp.addCourse(Course('Metodologia Científica e Tecnológica'          ,1,5))
engComp.addCourse(Course('Cálculo Numérico'                              ,2,5))
engComp.addCourse(Course('Banco de Dados'                                ,2,5))
engComp.addCourse(Course('Sistemas Lineares'                             ,2,5))
engComp.addCourse(Course('Sistemas Operacionais'                         ,2,5))
# 6 Semester
engComp.addCourse(Course('Engenharia de Software'                        ,2,6))
engComp.addCourse(Course('Probabilidade e Estatística'                   ,2,6))
engComp.addCourse(Course('Redes de Computadores'                         ,2,6))
engComp.addCourse(Course('Sistemas Embarcados'                           ,3,6))
# 7 Semester
engComp.addCourse(Course('Interação Humano-Computador'                   ,2,7))
engComp.addCourse(Course('Computação Gráfica'                            ,2,7))
engComp.addCourse(Course('Grafos'                                        ,2,7))
engComp.addCourse(Course('Produção Textual'                              ,1,7))
engComp.addCourse(Course('Introdução à Automação Industrial e Controle'  ,2,7))
# 8 Semester
engComp.addCourse(Course('Projeto de Sistemas de Informação'             ,2,8))
engComp.addCourse(Course('Inteligência Computacional'                    ,2,8))
engComp.addCourse(Course('Sistemas Distribuídos'                         ,1,8))
engComp.addCourse(Course('Sistemas de Tempo Real'                        ,1,8))
engComp.addCourse(Course('Aplicações de Controle'                        ,2,8))
# 9 Semester
engComp.addCourse(Course('Trabalho de Graduação Interdisciplinar'        ,1,9))
engComp.addCourse(Course('Empreendedorismo e Gestão'                     ,1,9))
engComp.addCourse(Course('Programação Paralela e Distribuída'            ,3,9))
engComp.addCourse(Course('Optativa 1'                                    ,2,9))
# 10 Semester
engComp.addCourse(Course('Ética e Filosofia'                            ,1,10))
engComp.addCourse(Course('Projeto Social'                               ,1,10))
engComp.addCourse(Course('Optativa 2'                                   ,2,10))
engComp.addCourse(Course('Optativa 3'                                   ,2,10))
print(engComp)

Program: EngComp (10 semesters, 45 courses, 0 classrooms, 0 teachers)


In [5]:
class Schedule:
    def __init__(self, program):
        self.weekName = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex']
        self.schedule = list([[[0 for z in range((int)(WEEK_CLASSES_SIZE/WEEK_SIZE))] for j in range(WEEK_SIZE)] for i in range(program.semesters)])
        self.generateRandomSchedule(program)
        program.reloadCoursesAmmo()
        
    def generateRandomSchedule(self, program):
        for i in range(1, program.semesters+1):
            history = []
            while(True):
                courses = program.getCoursesBySemester(i)
                if(courses == []):
                    break
                randomCourse = random.choice(courses)
                program.courses[program.courses.index(randomCourse)].ammo-=1
                #Put class on a random place
                while(True):
                    random1 = random.randint(1,WEEK_SIZE)
                    random2 = random.randint(1,(int)(WEEK_CLASSES_SIZE/WEEK_SIZE))
                    if (random1,random2) not in history:
                        history.append((random1,random2))
                        break
                self.schedule[i-1][random1-1][random2-1] = randomCourse.id
        return True
    
    def __repr__(self):
        strOut = '|  '
        for i in self.weekName:
            strOut += i + '  '
        strOut += '|\n'
        for i in range(len(self.schedule)):
            strOut += '|---------------------------|\n'
            if(i+1>=10):
                strOut += '|        ' + 'Semester ' + str(i+1) + '        |\n'
            else:
                strOut += '|        ' + 'Semester ' + str(i+1) + '         |\n'
            strOut += '|---------------------------|\n'
            strOut += '|  '
            for j in self.schedule[i]:
                if(j[0]>=10):
                    strOut += ''  + str(j[0]) + '   '
                else: 
                    strOut += ' ' + str(j[0]) + '   '
            strOut += '| AB\n'
            strOut += '|  '
            for j in self.schedule[i]:
                if(j[1]>=10):
                    strOut += ''  + str(j[1]) + '   '
                else: 
                    strOut += ' ' + str(j[1]) + '   '
            strOut += '| CD\n'
        return strOut

#Testes
s = Schedule(engComp)
print(s)


|  Seg  Ter  Qua  Qui  Sex  |
|---------------------------|
|        Semester 1         |
|---------------------------|
|   3    3    2    1    4   | AB
|   2    1    2    3    4   | CD
|---------------------------|
|        Semester 2         |
|---------------------------|
|   6    7    8    5    9   | AB
|   6    5    8    7    9   | CD
|---------------------------|
|        Semester 3         |
|---------------------------|
|  12   14   11   14   12   | AB
|  11   13   10   10   13   | CD
|---------------------------|
|        Semester 4         |
|---------------------------|
|  16   18   18   16   15   | AB
|  15   18   17   17   17   | CD
|---------------------------|
|        Semester 5         |
|---------------------------|
|   0   20   23   21   23   | AB
|  20   19   21   22   22   | CD
|---------------------------|
|        Semester 6         |
|---------------------------|
|  27   26   25   26   27   | AB
|  24   27   25   24    0   | CD
|---------------------------|
|   